In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
!pip install pycountry_convert
import pycountry
import pycountry_convert as pc
!pip install cufflinks chart_studio plotly
import chart_studio.plotly as py
import plotly.graph_objs as go
from plotly.offline import iplot, init_notebook_mode
import plotly.express as px
import cufflinks
cufflinks.go_offline(connected=True)
init_notebook_mode(connected=True)
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
train = pd.read_csv('../input/covid19-global-forecasting-week-5/train.csv',parse_dates=['Date'])
usdata = pd.read_csv('../input/jhu-covid19-data-with-us-state-data-prior-to-mar-9/covid19_train_data_us_states_before_march_09_new.csv',parse_dates=['Date'])
train.tail()
test_conducted =  pd.read_csv('../input/covid19-tests-conducted-by-country/Tests_conducted_05April2020.csv',parse_dates=['As of'])

In [ ]:
usdata.rename(columns={'Country.Region' : 'Country_Region' },inplace=True)
usdata.replace(['Burma','Korea, South','Taiwan*','US','Congo (Brazzaville)','Congo (Kinshasa)','West Bank and Gaza'],
               ['Myanmar','South Korea','Taiwan','United States','Congo','Congo','Gaza'],inplace=True)

In [ ]:
train.replace(['Burma','Korea, South','Taiwan*','US','Congo (Brazzaville)','Congo (Kinshasa)','West Bank and Gaza'],
               ['Myanmar','South Korea','Taiwan','United States','Congo','Congo','Gaza'],inplace=True
               )

In [ ]:
train.loc[(train.Date =='2020-05-01') & (train.Province_State.isnull())]

In [ ]:
def countrycode(x):
    try:
        return pc.country_name_to_country_alpha2(x)
    except:
        return "Other"
    
def continent(x):
    try:
        return pc.country_alpha2_to_continent_code(x)
    except:
        return "Other"

In [ ]:
train['country_code'] = train['Country_Region'].apply(countrycode)
train['continent'] = train['country_code'].apply(continent)
train.continent.replace(['AS', 'EU', 'AF', 'NA', 'SA', 'OC'],['Asia','Europe','Africa','North America','South America','Oceania'],inplace=True)

In [ ]:
day = train.Date.max()
summary = train[train.Province_State.isnull()].groupby(['continent','Country_Region','Target'])['TargetValue'].sum().reset_index()
summary = summary.pivot_table(index=['continent','Country_Region'],columns='Target',values='TargetValue').reset_index()
day_text = day.strftime('%B %d, %Y')

In [ ]:
continent_summary = summary.groupby(['continent'])['ConfirmedCases','Fatalities'].sum().reset_index()

In [ ]:
import plotly.express as px
from plotly.subplots import make_subplots
fig = make_subplots(rows=1, cols=2, specs=[[{'type':'domain'}, {'type':'domain'}]])
fig.add_trace(go.Pie(labels=continent_summary.continent, values=continent_summary.ConfirmedCases, name="Confirmed Cases"),
              1, 1)
fig.add_trace(go.Pie(labels=continent_summary.continent, values=continent_summary.Fatalities, name="Fatalities"),
              1, 2)
fig.update_traces(hole=.4, hoverinfo="label+value")
fig.update_layout(
    title_text="Covid Spread across Continents on " +str(day_text),
    # Add annotations in the center of the donut pies.
    annotations=[dict(text='Confirmed Cases', x=0.15, y=0.5, font_size=12, showarrow=False),
                 dict(text='Fatalities', x=0.82, y=0.5, font_size=12, showarrow=False)])
fig.show()

In [ ]:
summary[summary.ConfirmedCases>100].iplot(x='ConfirmedCases',y='Fatalities',categories='continent',text='Country_Region',
              layout=dict(yaxis=dict(type='log',tickvals=[1,10,100,1000,10000], title='Fatalities'),
                          xaxis=dict(type='log',tickvals=[1,10,100,1000,10000,100000],title='Confirmed Cases'),
                title='Country wide Status of Covid Cases(min: of 100 reported cases) on ' + str(day_text)))


In [ ]:
from textwrap import wrap
def statusplothorgrid(a,b,c, **kwargs):
    ax = plt.gca()
    #data = kwargs.pop("data")
    plot=plt.barh(x=a,y=[b,c],ax=ax, grid=False, **kwargs)
    labels = [item.get_text() for item in plot.get_yticklabels()]
    labels =[ '\n'.join(wrap(l, 15)) for l in labels ]
    plot.set_yticklabels(labels)
    for p in plot.patches:
             plot.annotate("%g" % p.get_width(), ( p.get_width(),p.get_y()+ p.get_height()/2),
                 ha='center', va='center', fontsize=12, color='black', xytext=(20,0),
                 textcoords='offset points') 
    plot.set(ylabel='',xlabel="")

In [ ]:
# top20= summary.sort_values('ConfirmedCases',ascending= False).groupby('continent').head(20).reset_index()
# g = sns.FacetGrid(top20, col="continent",col_wrap=2,sharex=False,sharey=False,height=9,aspect=1.5)
# g = (g.map(statusplothorgrid,'Country_Region','ConfirmedCases','Fatalities').set_titles("{col_name}",size=24))

In [ ]:
def statusplot(x,y,ax1,ch=False,degree=0):
    plotdata= summary.loc[(summary.ConfirmedCases <= x) & (summary.ConfirmedCases >= y)].sort_values('ConfirmedCases', ascending=False)
    plot1=plotdata.plot(x='Country_Region',y=['ConfirmedCases','Fatalities'],kind='bar',rot=degree,legend = ch,ax=ax1)
    for p in plot1.patches:
        h = p.get_height()
        x = p.get_x()+p.get_width()/2.
        if h != 0:
            plot1.annotate("%g" % p.get_height(), xy=(x,h), xytext=(0,4),rotation=90, 
                           textcoords="offset points", ha="center", va="bottom")
    plot1.set(ylabel='',xlabel="")
    

In [ ]:
from textwrap import wrap
def statusplothor(x,y,ax1):
    plotdata= summary.loc[(summary.ConfirmedCases <= x) & (summary.ConfirmedCases >= y)].sort_values('ConfirmedCases', ascending=True)
    plot=plotdata.plot.barh(x='Country_Region',y=['ConfirmedCases','Fatalities'],ax=ax1)
    labels = [item.get_text() for item in plot.get_yticklabels()]
    labels =[ '\n'.join(wrap(l, 15)) for l in labels ]
    plot.set_yticklabels(labels)
    for p in plot.patches:
             plot.annotate("%g" % p.get_width(), ( p.get_width(),p.get_y()+ p.get_height()/2),
                 ha='center', va='center', fontsize=12, color='black', xytext=(20,0),
                 textcoords='offset points') 
    plot.set(ylabel='',xlabel="")
    

**Country wide Status as bar plot**

For better visulaization the count of country wide counts of Confirmed Cases and Fatalities are plotted below in the descending order of reported cases.

In [ ]:
np.warnings.filterwarnings('ignore')
upper= summary.ConfirmedCases.max() +100
day = train.Date.max()
#sns.barplot(data=summary.sort_values('ConfirmedCases', ascending=False)[:50],x='Country_Region',y='ConfirmedCases')
#plotdata= summary.sort_values('ConfirmedCases', ascending=False)[:20]
fig, axs = plt.subplots(2,2, figsize=(16,45), facecolor='w', edgecolor='k',squeeze=False)
axs=axs.ravel()
statusplothor(upper,25000,axs[0])
statusplothor(25000,5000,axs[1])
statusplothor(5000,700,axs[2])
statusplothor(700,50,axs[3])
fig.suptitle('Global Status of Covid cases as on ' + str(day_text))
fig.tight_layout(pad=0.4, w_pad=0.5, h_pad=1.0)
fig.subplots_adjust(top=0.96)

In [ ]:
from datetime import datetime,timedelta
Previous_Day = day - timedelta(days=1)

In [ ]:
train_country = train[train.Province_State.isnull()]
train_country = train_country.pivot_table(index=['Country_Region', 'Population',
       'Weight', 'Date'],columns='Target',values='TargetValue').reset_index()

In [ ]:
train_country['Fatalities'].fillna(0,inplace=True)
train_country['ConfirmedCases'].fillna(0,inplace=True)
train_country[train_country.Date>'2020-05-01'].head()

In [ ]:
#usdatagroup = usdata.groupby(['Country_Region','Date'])['ConfirmedCases','Fatalities'].sum().reset_index()

In [ ]:
data = train_country.groupby(['Country_Region','Date'])['ConfirmedCases','Fatalities'].sum().reset_index()

In [ ]:
data['cum_ConfirmedCases'] = data.groupby(['Country_Region'])['ConfirmedCases'].transform(pd.Series.cumsum)
data['cum_Fatalities'] = data.groupby(['Country_Region'])['Fatalities'].transform(pd.Series.cumsum)

In [ ]:
#data = pd.concat([data,usdatagroup]).drop_duplicates(['Country_Region','Date'],keep='last').sort_values(['Country_Region','Date'])

In [ ]:
list1= ['United States','Spain','Italy','Germany','France','United Kingdom','Korea, South','Iran','Israel','Japan','India']
limitdata = data[(data['Country_Region'].isin(list1))& (data.cum_ConfirmedCases>1000) & (data.cum_ConfirmedCases<65000)]
g= limitdata.groupby('Country_Region')
(pd.concat([g.head(1), g.tail(1)]).drop_duplicates().sort_values('Country_Region').reset_index(drop=True))

In [ ]:
def plotdaynum(x,y):
    plotdata = data[(data.cum_ConfirmedCases>x) & (data.cum_ConfirmedCases<y)]
    plotdata['Daynum'] = plotdata.groupby('Country_Region').cumcount() + 1
    plotdata=plotdata.sort_values('cum_ConfirmedCases',ascending= False).groupby('Country_Region').head(1).reset_index()
    return plotdata
    

In [ ]:
plotdata = plotdaynum(500,65000)
plotdata[plotdata.cum_ConfirmedCases>50000].iplot(x='Daynum',y='cum_ConfirmedCases',categories='Country_Region',text='Country_Region',
              layout=dict(yaxis=dict(title='Confirmed Cases'),
                          xaxis=dict(tickvals=[1,5,10,15,20,25,30,35,40,45,50,55,60],title='Number of days upto 65,000 cases from day of 500 Confirmed Cases'),
                title='How countries have succeded in delaying Covid spread (min 50000 cases)'))

In [ ]:
sns.set(rc={'figure.figsize':(12,8)})
colors = ["windows blue", "amber", "greyish", "faded green", "dusty purple","pale red",'light blue','black']
plt.subplot(2,1,1)
plotdata = data[data['Country_Region'].isin(['United States','Spain','Italy','United Kingdom','Russia','Germany','France','India'])]
plotdata = plotdata[(plotdata.cum_ConfirmedCases>500) & (plotdata.cum_ConfirmedCases<65000)]
plotdata['Daynum'] = plotdata.groupby('Country_Region').cumcount() + 1
plot1 = sns.lineplot(x='Daynum',y='cum_ConfirmedCases',hue='Country_Region',data= plotdata,palette=sns.xkcd_palette(colors))
plot1.set(xlabel="")
plt.subplot(2,1,2)
plotdata = data[data['Country_Region'].isin(['South Korea','Iran','Israel','Japan','India','Turkey','Brazil','Peru'])]
plotdata = plotdata[(plotdata.cum_ConfirmedCases>500) & (plotdata.cum_ConfirmedCases<65000)]
plotdata['Daynum'] = plotdata.groupby('Country_Region').cumcount() + 1
plot2 = sns.lineplot(x='Daynum',y='cum_ConfirmedCases',hue='Country_Region',data= plotdata,palette=sns.xkcd_palette(colors))
plot2.set(xlabel="Number of Days since 500th reported case")
plt.suptitle('Analysis of rate of increase of first 500 to 65000 reported Covid Cases across major Countries ')

**Daily Increase in Covid cases in Major Countries**

In [ ]:
sns.set(rc={'figure.figsize':(16,12)})
#plt.subplot(2,1,1)
plotdata = data[data['Country_Region'].isin(['United States','Spain','Italy','Germany','France','United Kingdom','Russia'])]
plotdata = plotdata[(plotdata.cum_ConfirmedCases>1000)]
plotdata['Daynum'] = plotdata.groupby('Country_Region').cumcount() + 1
#plotdata['Day_increase'] = plotdata.groupby('Country_Region')['ConfirmedCases'].diff()
g = sns.FacetGrid(plotdata, col="Country_Region",col_wrap=2,sharex=False,sharey=False,height=9,aspect=1.5)
g = (g.map(sns.barplot,'Daynum','ConfirmedCases').set_titles("{col_name}",size=24))


In [ ]:
sns.set(rc={'figure.figsize':(16,12)})
#plt.subplot(2,1,1)
plotdata = data[data['Country_Region'].isin(['South Korea','Iran','Israel','Japan','India','Turkey'])]
plotdata = plotdata[(plotdata.cum_ConfirmedCases>1000)]
plotdata['Daynum'] = plotdata.groupby('Country_Region').cumcount() + 1
#plotdata['Day_increase'] = plotdata.groupby('Country_Region')['ConfirmedCases'].diff()
g = sns.FacetGrid(plotdata, col="Country_Region",col_wrap=2,sharex=False,sharey=False,height=9,aspect=1.5)
g = (g.map(sns.barplot,'Daynum','ConfirmedCases').set_titles("{col_name}",size=24))

In [ ]:
sns.set(rc={'figure.figsize':(16,12)})
#plt.subplot(2,1,1)
plotdata = data[data['Country_Region'].isin(['Brazil','Peru'])]
plotdata = plotdata[(plotdata.cum_ConfirmedCases>1000)]
plotdata['Daynum'] = plotdata.groupby('Country_Region').cumcount() + 1
g = sns.FacetGrid(plotdata, col="Country_Region",col_wrap=2,sharex=False,sharey=False,height=9,aspect=1.5)
g = (g.map(sns.barplot,'Daynum','ConfirmedCases').set_titles("{col_name}",size=24))